# Mohammad Amin Rami
# HW3
# 98101588

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import random

In [2]:
spark = SparkSession \
    .builder \
    .appName("Spark Processor") \
    .master("local[*]") \
    .getOrCreate()

sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/23 22:19:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Part A

In [3]:
data_df = spark.read.csv('Sample_Data/Sample_Traffic.csv', header=True)
data_df.show(10)

+-----------+---------+---------------+-------------+----------------+----------+-------------------+
|DEVICE_CODE|SYSTEM_ID|ORIGINE_CAR_KEY|FINAL_CAR_KEY|CHECK_STATUS_KEY|COMPANY_ID|      PASS_DAY_TIME|
+-----------+---------+---------------+-------------+----------------+----------+-------------------+
|     200501|       81|       10477885|     10477885|               5|       161|2021-06-01 03:54:39|
|        155|       81|       87625017|     87625017|               5|       161|2021-06-01 04:14:21|
|     631757|       81|        8652928|      8652928|               5|       161|2021-06-01 03:58:57|
|     631757|       81|        8548123|      8548123|               5|       161|2021-06-01 04:01:38|
|     631757|       81|       24715264|     24715264|               5|       161|2021-06-01 03:56:57|
|     206602|       81|       69939810|     69939810|               5|       161|2021-06-01 04:06:38|
|     206602|       81|       11046172|     11046172|               5|       161|2

In [4]:
# removing outliers
data_df = data_df.filter(data_df['ORIGINE_CAR_KEY'] == data_df['FINAL_CAR_KEY'])

In [5]:
# create device codes rdd
device_codes = data_df.select('DEVICE_CODE').distinct().rdd.map(lambda x: x['DEVICE_CODE']).sortBy(lambda x: int(x))

In [6]:
device_codes_dict = sc.broadcast({code: idx for idx, code in enumerate(device_codes.collect())})

In [7]:
def format_date(date):
    date = list(date)
    date[-2:] = '{:02d}'.format(int(''.join(date[-2:]))).split()
    return ''.join(date)

def create_key_value(example):
    key = (example['FINAL_CAR_KEY'], format_date(example['PASS_DAY_TIME'][:-3]))
    value = example['DEVICE_CODE']
    return (key, value)

data_rdd = data_df.rdd.map(create_key_value)
data_rdd.take(20)

[(('10477885', '2021-06-01 03:54'), '200501'),
 (('87625017', '2021-06-01 04:14'), '155'),
 (('8652928', '2021-06-01 03:58'), '631757'),
 (('8548123', '2021-06-01 04:01'), '631757'),
 (('24715264', '2021-06-01 03:56'), '631757'),
 (('69939810', '2021-06-01 04:06'), '206602'),
 (('11046172', '2021-06-01 04:03'), '206602'),
 (('8334658', '2021-06-01 04:09'), '206602'),
 (('29077699', '2021-06-01 04:11'), '206602'),
 (('40682798', '2021-06-01 03:54'), '206602'),
 (('22474283', '2021-06-01 04:06'), '206602'),
 (('8091143', '2021-06-01 03:57'), '206602'),
 (('28889922', '2021-06-01 03:59'), '206602'),
 (('21720169', '2021-06-01 04:00'), '206602'),
 (('48823778', '2021-06-01 04:00'), '206602'),
 (('52789835', '2021-06-01 04:04'), '206602'),
 (('10919200', '2021-06-01 04:03'), '631829'),
 (('13177831', '2021-06-01 03:51'), '631829'),
 (('101646284', '2021-06-01 02:28'), '202901'),
 (('30889141', '2021-06-01 03:07'), '202901')]

In [8]:
def flatten(lis):
    if isinstance(lis, list):
        for item in lis:
            yield from flatten(item)
    else:
        yield lis

In [9]:
# here we create the (basket, [items]) rdd
baskets = data_rdd.groupBy(lambda x: x[0]).mapValues(lambda x: list({item[1] for item in list(x)}))
baskets.take(20)

[(('10477885', '2021-06-01 03:54'), ['200501']),
 (('8548123', '2021-06-01 04:01'), ['631757']),
 (('8334658', '2021-06-01 04:09'), ['206602']),
 (('22474283', '2021-06-01 04:06'), ['206602']),
 (('19924252', '2021-06-01 02:30'), ['101301']),
 (('32421847', '2021-06-01 04:07'), ['230204']),
 (('58953919', '2021-06-01 03:19'), ['631362']),
 (('8550078', '2021-06-01 03:44'), ['631829']),
 (('35081338', '2021-06-01 03:45'), ['205201']),
 (('7949037', '2021-06-01 04:07'), ['145']),
 (('8358998', '2021-06-01 01:51'), ['631771']),
 (('7657114', '2021-06-01 03:52'), ['202901']),
 (('11361805', '2021-06-01 01:44'), ['22010031']),
 (('21790369', '2021-06-01 01:31'), ['100701251']),
 (('9823557', '2021-06-01 03:52'), ['22010052', '22010051']),
 (('61614929', '2021-06-01 03:58'), ['203902']),
 (('35432386', '2021-06-01 02:11'), ['22009977']),
 (('24498245', '2021-06-01 02:35'), ['22009977']),
 (('8704382', '2021-06-01 02:55'), ['22009977']),
 (('89895041', '2021-06-01 04:11'), ['22010062'])]

In [10]:
baskets = baskets.groupBy(lambda x: (x[0][0], x[0][1].split(' ')[0])).mapValues(lambda x: list(flatten([item[1] for item in list(x)])))
baskets.take(20)

[(('15757536', '2021-06-01'),
  ['213402',
   '22009972',
   '900247',
   '100700845',
   '213202',
   '900164',
   '146',
   '206602',
   '22009971',
   '103002',
   '22009972']),
 (('76823072', '2021-06-01'),
  ['143',
   '900240',
   '100701059',
   '900234',
   '900108',
   '900171',
   '205802',
   '900212',
   '900244',
   '900246']),
 (('23125194', '2021-06-01'), ['210602']),
 (('7911538', '2021-06-01'), ['163']),
 (('34504614', '2021-06-01'),
  ['212802', '137', '900240', '22010118', '143', '900256', '205802']),
 (('46522958', '2021-06-01'),
  ['137', '205802', '900149', '900240', '900234', '22010118']),
 (('20751398', '2021-06-01'),
  ['200301',
   '900255',
   '900236',
   '100700812',
   '115',
   '100700910',
   '100700841',
   '900159',
   '900135',
   '231',
   '900246',
   '631367',
   '900107',
   '900237']),
 (('14327784', '2021-06-01'), ['211401']),
 (('27628070', '2021-06-01'), ['203101']),
 (('11245361', '2021-06-01'),
  ['631346',
   '900133',
   '900259',
   '9002

In [11]:
# deleting outliers
baskets = baskets.filter(lambda x: len(x[1]) <= 300)

In [12]:
baskets.take(10)

[(('15757536', '2021-06-01'),
  ['213402',
   '22009972',
   '900247',
   '100700845',
   '213202',
   '900164',
   '146',
   '206602',
   '22009971',
   '103002',
   '22009972']),
 (('76823072', '2021-06-01'),
  ['143',
   '900240',
   '100701059',
   '900234',
   '900108',
   '900171',
   '205802',
   '900212',
   '900244',
   '900246']),
 (('23125194', '2021-06-01'), ['210602']),
 (('7911538', '2021-06-01'), ['163']),
 (('34504614', '2021-06-01'),
  ['212802', '137', '900240', '22010118', '143', '900256', '205802']),
 (('46522958', '2021-06-01'),
  ['137', '205802', '900149', '900240', '900234', '22010118']),
 (('20751398', '2021-06-01'),
  ['200301',
   '900255',
   '900236',
   '100700812',
   '115',
   '100700910',
   '100700841',
   '900159',
   '900135',
   '231',
   '900246',
   '631367',
   '900107',
   '900237']),
 (('14327784', '2021-06-01'), ['211401']),
 (('27628070', '2021-06-01'), ['203101']),
 (('11245361', '2021-06-01'),
  ['631346',
   '900133',
   '900259',
   '9002

In [13]:
def binary_vector(row):
    v = np.zeros(len(device_codes_dict.value))
    indices = [device_codes_dict.value[code] for code in row[1]]
    v[indices] = 1
    return *row, v

binary_vectors = baskets.map(binary_vector)

### Part B

In [14]:
def cosine_similarity(v1, v2):
    dot_prod = np.sum(v1 * v2)
    sim = np.arctan2(dot_prod, np.linalg.norm(v1) * np.linalg.norm(v2))
    return sim

In [15]:
path_len = 3

query_path = np.random.choice(list(device_codes_dict.value.keys()), size=path_len)
query_path = list(set(query_path))

query_path_binary_vector = np.zeros(len(device_codes_dict.value))
query_path_binary_vector[[device_codes_dict.value[code] for code in query_path]] = 1
query_path_binary_vector = sc.broadcast(query_path_binary_vector)

In [16]:
def query_similarity(row):
    similarity = cosine_similarity(query_path_binary_vector.value, row[2])
    return (row[0], row[1], similarity)


similarities = binary_vectors.map(query_similarity)

In [17]:
most_similar_paths = similarities.top(5, key=lambda x: x[2])

print('===== Resluts =====')
print('The query:')
print(list(query_path))
print('Most similar paths:')
for i, path in enumerate(most_similar_paths):
    print(f'{i+1}- {path}')

===== Resluts =====
The query:
['1001079', '22009923', '22010119']
Most similar paths:
1- (('95985673', '2021-06-01'), ['22009923', '22010119'], 0.6847192030022828)
2- (('23214739', '2021-06-01'), ['22009923', '22010119'], 0.6847192030022828)
3- (('8075171', '2021-06-01'), ['22009923', '22010119', '100700965'], 0.5880026035475676)
4- (('17974480', '2021-06-01'), ['900236', '22010119', '22009923'], 0.5880026035475676)
5- (('17122136', '2021-06-01'), ['100700824', '22010119', '22009923', '100700824'], 0.5880026035475676)


### Part C

In [18]:
r = 10
b = 5
n_hashes = r * b
b = sc.broadcast(b)
r = sc.broadcast(r)
hyperplanes = [np.random.choice([-1, 1], len(device_codes_dict.value)) for _ in range(n_hashes)]
hyperplanes = sc.broadcast(hyperplanes)

In [19]:
def sign(x):
    return 1.0 if x>=0 else -1 

In [20]:
query_path_signature = [sign(sum(query_path_binary_vector.value * hyperplane)) for hyperplane in hyperplanes.value]
query_path_signature = sc.broadcast(query_path_signature)

In [21]:
# generate signature matrix
def generate_signature(row):
    signature = np.zeros(len(hyperplanes.value))
    for i, hp in enumerate(hyperplanes.value):
        signature[i] = sign(sum(hp * row[2]))
    return *row, signature

signatures = binary_vectors.map(generate_signature)

In [22]:
def find_candidates(row):
    is_candidate = False
    for k in range(b.value):
        indexes = slice(k*r.value, (k+1)*r.value)
        if all(row[3][indexes] == query_path_signature.value[indexes]):
            is_candidate = True
            break
    return *row, is_candidate
    
rows_with_candidates = signatures.map(find_candidates)

In [23]:
candidates = rows_with_candidates.filter(lambda x: x[4])

In [35]:
def find_similarity(row):
    sim = cosine_similarity(row[2], query_path_binary_vector.value)
    return *row, sim
    

In [36]:
top_candidates = candidates.map(find_similarity).top(5, key=lambda x: x[5])

In [39]:
top_candidates = [(candidate[0], candidate[1], candidate[-1]) for candidate in top_candidates]

In [41]:
print('===== Resluts =====')
print('The query:')
print(list(query_path))
print('Most similar paths:')
for i, path in enumerate(top_candidates):
    print(f'{i+1}- {path}')

===== Resluts =====
The query:
['1001079', '22009923', '22010119']
Most similar paths:
1- (('95985673', '2021-06-01'), ['22009923', '22010119'], 0.6847192030022828)
2- (('23214739', '2021-06-01'), ['22009923', '22010119'], 0.6847192030022828)
3- (('8075171', '2021-06-01'), ['22009923', '22010119', '100700965'], 0.5880026035475676)
4- (('17974480', '2021-06-01'), ['900236', '22010119', '22009923'], 0.5880026035475676)
5- (('17122136', '2021-06-01'), ['100700824', '22010119', '22009923', '100700824'], 0.5880026035475676)
